### Import packages

In [152]:
import pandas as pd

### Create pandas dataframe by reading csv file

In [153]:
df = pd.read_csv('medals.csv')

### How does data look in general?

In [154]:
df.head(10)

,Games,Year,Sport,Discipline,Athlete,Team,Gender,Event,Medal,Gold,Silver,Bronze
0,Athens (1896),1896,Aquatics,Swimming,alfred hajos,HUN,Men,100m freestyle men,Gold,1,0,0
1,Athens (1896),1896,Aquatics,Swimming,Otto Herschmann,AUT,Men,100m freestyle men,Silver,0,1,0
2,Athens (1896),1896,Aquatics,Swimming,dimitrios drivas,GRE,Men,100m freestyle for sailors men,Bronze,0,0,1
3,Athens (1896),1896,Aquatics,Swimming,Ioannis Malokinis,GRE,Men,100m freestyle for sailors men,Gold,1,0,0
4,Athens (1896),1896,Aquatics,Swimming,spiridon chasapis,GRE,Men,100m freestyle for sailors men,Silver,0,1,0
5,Athens (1896),1896,Aquatics,Swimming,Efstathios Chorophas,GRE,Men,1200m freestyle men,Bronze,0,0,1
6,Athens (1896),1896,Aquatics,Swimming,alfred hajos,HUN,Men,1200m freestyle men,Gold,1,0,0
7,Athens (1896),1896,Aquatics,Swimming,joannis andreou,GRE,Men,1200m freestyle men,Silver,0,1,0
8,Athens (1896),1896,Aquatics,Swimming,Efstathios Chorophas,GRE,Men,400m freestyle men,Bronze,0,0,1
9,Athens (1896),1896,Aquatics,Swimming,paul neumann,AUT,Men,400m freestyle men,Gold,1,0,0


### Is data missing?

In [155]:
df.isnull().sum()

Games         0
Year          0
Sport         0
Discipline    0
Athlete       0
Team          0
Gender        0
Event         0
Medal         0
Gold          0
Silver        0
Bronze        0
dtype: int64

$\rightarrow$ no entry is NaN

### Are Sport column and Discipline column the same for Athletics?

In [156]:
# get all results where Sport=='Athletics' or Discipline=='Athletics' and get unique Sport
pd.unique(df[(df.Sport=='Athletics') | (df.Discipline=='Athletics')].Sport)

array(['Athletics'], dtype=object)

In [157]:
# get all results where Sport=='Athletics' or Discipline=='Athletics' and get unique Discipline
pd.unique(df[(df.Sport=='Athletics') | (df.Discipline=='Athletics')].Discipline)

array(['Athletics'], dtype=object)

### Is gold medal data 0/1 encoding consistent?

In [158]:
pd.unique(df.Gold)

array([ 1,  0, 10])

In [159]:
df[df.Gold==10]

,Games,Year,Sport,Discipline,Athlete,Team,Gender,Event,Medal,Gold,Silver,Bronze
16927,Los Angeles (1984),1984,Athletics,Athletics,Carl Lewis,USA,Men,long jump men,Gold,10,0,0


$\rightarrow$ no, fix entry with Gold=1 to not falsify aggregate 

In [160]:
df.at[16927, 'Gold']  = 1
df.iloc[16927]

Games         Los Angeles (1984)
Year                        1984
Sport                  Athletics
Discipline             Athletics
Athlete               Carl Lewis
Team                         USA
Gender                       Men
Event              long jump men
Medal                       Gold
Gold                           1
Silver                         0
Bronze                         0
Name: 16927, dtype: object

In [161]:
print(pd.unique(df.Medal))

['Gold' 'Silver' 'Bronze']


$\rightarrow$ consistent, can be used

is data correct for other medals?

In [162]:
print(pd.unique(df.Silver))
print(pd.unique(df.Bronze))

[0 1]
[0 1]


In [163]:
# are there entries with multiple medals?
print((df.Gold + df.Silver + df.Bronze).unique())

[1]


$\rightarrow$ consistent

In [164]:
# is the column medal consistent with the entries in the columns Gold, Silver, Bronze? 

In [165]:
pd.unique((df.Medal == 'Gold') == (df.Gold == 1))

array([ True])

In [166]:
pd.unique((df.Medal == 'Silver') == (df.Silver == 1))

array([ True])

In [167]:
pd.unique((df.Medal == 'Bronze') == (df.Bronze == 1))

array([ True])

$\rightarrow$ consistent 

### Are there duplicates in the data? 

In [168]:
# Check if there are duplicates (If True then there are duplicates)
df.duplicated().any()

True

In [169]:
# Remove duplicated rows
df = df.drop_duplicates()
# df[df.Athlete=='Carl Lewis']

### Question 1: Find the five athletes with the most gold medals in athletics

In [170]:
# select all rows of gold medalists in athletics, group them by athlete's name and sort
# descending by the sum of their gold medals
athletics_medalists_sorted = df[(df.Sport=='Athletics') & (df.Medal=='Gold')][['Athlete','Gold']] \
        .groupby(['Athlete']) \
        .sum() \
        .sort_values(by = ['Gold'],ascending=[False]) \
        .reset_index()

In [171]:
athletics_medalists_sorted[:5]

,Athlete,Gold
0,Carl Lewis,9
1,Usain Bolt,9
2,Paavo Nurmi,9
3,Allyson Felix,6
4,Ville Ritola,5


### Question 2: Find the 5 athletes with gold medals in the largest number of (different) athletics events 

In [172]:
df.Event = df.Event.str.replace(' women','')
df.Event = df.Event.str.replace(' men','')

In [173]:
# select all rows of gold medalists, group them by athlete's name and sort descending by the sum of their unique Events 
# reset indices to not use atheletes' names 
events_medalists_sorted = df[(df.Medal=='Gold') & (df.Discipline=='Athletics')].groupby(['Athlete']) \
        .agg({'Event': pd.Series.nunique}) \
        .sort_values(by = ['Event'],ascending=[False])  \
        .reset_index()


events_medalists_sorted[:5] 

,Athlete,Event
0,Paavo Nurmi,6
1,Ville Ritola,5
2,Jesse Owens,4
3,Fanny Blankers-koen,4
4,Alvin Kraenzlein,4


### Question 3: If the olympics committee found guilty of doping the first and the third of the five athletes from question2 create a new medals.csv dataset with redistributed medals after the gold medals of those two athletes are removed. Who then are the top five that won gold medals in the largest number of different athletic events?

In [174]:
# list of athletes to consider 
guilty_athletes = list(events_medalists_sorted[0:1].Athlete) + list(events_medalists_sorted[2:8].Athlete) 
print(guilty_athletes)

['Paavo Nurmi', 'Jesse Owens', 'Fanny Blankers-koen', 'Alvin Kraenzlein', 'Carl Lewis', 'Betty Cuthbert', 'Hannes Kolehmainen']


In [175]:
# events to consider
print(pd.unique(df[df.Athlete.isin(guilty_athletes)].Games))

['Paris (1900)' 'Stockholm (1912)' 'Antwerp (1920)' 'Paris (1924)'
 'Amsterdam (1928)' 'Berlin (1936)' 'London (1948)'
 'Melbourne / Stockholm (1956)' 'Tokyo (1964)' 'Los Angeles (1984)'
 'Seoul (1988)' 'Barcelona (1992)' 'Atlanta (1996)']


Assumption: If an athlete in a team sport event (relay for instance) is disqualified, the whole team is then disqualified <br/>
The new bronze medals will be filled with Athlete='Unknown' and Team='Unknown'

In [176]:
def adjust_event(games, event, medal, gender):
        # select relevant event rows
        event_df = df[(df.Games == games) & (df.Event == event) & (df.Gender == gender)]

        medals = ['Gold','Silver','Bronze']
        
        # iterate over medals to adjust
        while len(medals) > 0:
            # remove medal from list
            m = medals.pop(0)
            # get indices corresponding to medal
            idx = event_df[event_df.Medal==m].index.tolist()

            # if medal=='Bronze', add unknown Athletes and Team
            if len(medals)==0 :
                names, teams = ['Unknown']*len(idx), ['Unknown']*len(idx)
            # else use Athletes of place below
            else:
                names, teams = event_df[event_df.Medal==medals[0]].Athlete.tolist(), event_df[event_df.Medal==medals[0]].Team.tolist()

            # update dataframe, iterate over lists in case of team sport events
            for (i,n,t) in zip(idx, names, teams):
                df.at[i, 'Athlete']  = n
                df.at[i, 'Team']  = t

Iteratively overwrite rows in question:

In [177]:
# for each of the guilty athletes
for athlete in guilty_athletes:
    # find all the events they won a gold medal in
    results = df[(df.Athlete==athlete) & (df.Medal=='Gold')][['Games','Event','Medal','Gender']]

    # for each event the offender has taken part in
    for index, row in results.iterrows():
        # adjust the scores
        adjust_event(row['Games'], row['Event'], row['Medal'], row['Gender'])

New 5 athletes with the most gold medals in different events:

In [178]:
#events_medalists_sorted_adjusted = df[(df.Medal=='Gold') & (df.Discipline=='Athletics')][['Athlete','Event','Gold']] \
 #   .groupby(['Athlete']) \
 #   .sum() \
 #   .sort_values(by = ['Gold'],ascending=[False]) \
 #   .reset_index()

events_medalists_sorted_adjusted = df[(df.Medal=='Gold') & (df.Discipline=='Athletics')].groupby(['Athlete']) \
        .agg({'Event': pd.Series.nunique}) \
        .sort_values(by = ['Event'],ascending=[False])  \
        .reset_index()

In [179]:
events_medalists_sorted_adjusted[:5] # 17 tied for second place

,Athlete,Event
0,Ville Ritola,4
1,Walter B. John Tewksbury,3
2,Renate Stecher,3
3,Harrison Dillard,3
4,Wilma Rudolph,3


Save adjusted data to csv, with ',' comma as separator as in the provided file:

In [180]:
df.to_csv('medals_adjusted.csv', sep=',')